In [1]:
#from utils import grab_screen, key_check
import cv2
import time
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using TensorFlow backend.


In [2]:
from directkeys import PressKey,ReleaseKey, W, A, S, D
from utils import *

In [3]:
t_time = 0.11

def straight():
##    if random.randrange(4) == 2:
##        ReleaseKey(W)
##    else:
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    
def left():
    PressKey(W)
    PressKey(A)
    #ReleaseKey(W)
    ReleaseKey(D)
    #ReleaseKey(A)
    time.sleep(t_time)
    ReleaseKey(A)
    #ReleaseKey(W)

def right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    #ReleaseKey(W)
    #ReleaseKey(D)
    time.sleep(t_time)
    ReleaseKey(D)
    #ReleaseKey(W)

In [4]:
model_path = 'D:/ML_DATA/wheelai/gtaV/results/'


In [6]:
def vgg_conv():
    model = VGG16(include_top=False, weights='imagenet', input_shape=(160,80,3))
    #remove the last convolutional block
    [model.layers.pop() for _ in range(4)]
    return model


def top_layer(input_shape):
    model = Sequential()
    model.add(Conv2D(512, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Dropout(0.2))
    #model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='elu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    return model 

In [7]:
import h5py
f = h5py.File(model_path+'vgg_ft.h5', 'r+')
del f['optimizer_weights']
f.close()

In [8]:
vgg_model = load_model(model_path+'vgg_ft.h5')

In [9]:
vgg_model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=2e-5, momentum=0.9), metrics=['accuracy'])

#image = grab_screen(region=(10, 450, 1500, 300))
path00 = 'C:/ML_DATA/wheelai/kerakart/valid/right/right1.jpg'
img = cv2.imread(path00)
img = cv2.resize(img,(80,160))
x = image.img_to_array(img)

In [10]:
def main():
    count_down(5)
    paused = False
    while(True):

        if not paused:
            
            img = grab_screen(region=(0, 80, 1280, 960))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img,(224,224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            action = vgg_model.predict(x, batch_size=1,verbose=0)[0]
            
            #print(action)
            
            turn_thresh = .75
            fwd_thresh = 0.70

            if action[1] > fwd_thresh:
                straight()
            elif action[0] > turn_thresh:
                left()
            elif action[2] > turn_thresh:
                right()
            else:
                straight()

        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)


In [ ]:
main()

5
4
3
2
1
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
Pausing!
unpaused!
